In [1]:
from dotenv import load_dotenv
load_dotenv()
import os 
openapi_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_core.tools import tool

@tool
def add(a: int, b:int):
    """adds a and b .
    Args : 
    a: first int
    b : second int
    """

    return a+b
@tool
def multiply(a:int , b:int):
    """Multiply a and b .
    Args :
    a: first int
    b: second int
    """
    return a*b
tools = [add, multiply]

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field

class add(BaseModel):
    """Add ."""
    a: int= Field(...,description='First integer :')
    b: int= Field(..., descripthin="Second integer :")


class multiply(BaseModel):
    """Multiply ."""

    a : int= Field(...,description="First integer :")
    b : int= Field(...,description="Second integer :")
    
tools = [add , multiply]

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")

In [7]:
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")

In [8]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_PDfifRXseDqGwPkhMnRn3gR0',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_R196Rsz9z9XRekSr81p3IulF',
  'type': 'tool_call'}]

In [9]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]

In [10]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_9zNWn2gdUbSTc80LdvEiylrG', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'add', 'args': '', 'id': 'call_crLjxaOW5BUyuxTOLsjQxJr5', 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[]


In [11]:
first = True

async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False

    else :
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_FutFqw9xyYnN7xRxdMyFCg2O', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a"', 'id': 'call_FutFqw9xyYnN7xRxdMyFCg2O', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, ', 'id': 'call_FutFqw9xyYnN7xRxdMyFCg2O', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_FutFqw9xyYnN7xRxdMyFCg2O', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_FutFqw9xyYnN7xRxdMyFCg2O', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_FutFqw9xyYnN7xRxdMyFCg2O', 'index': 0, 'type': 'tool_call_chunk'}, {'name': 'add', 'args': '', 'id': 'call_fup8kkf6lQpnudEZwUUYD71W', 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_FutFqw9xyYnN7xRxdMyFCg2O', 'index': 0, 'type': 'tool_call_chunk'}, {'name': 'add', 'arg

In [12]:
print(type(gathered.tool_call_chunks[0]['args']))

<class 'str'>


In [15]:
first = False
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = True
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

[{'name': 'multiplymultiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_6LhRibqh52pMYWDp6f2Wfy4Qcall_erQMMzwDObuV2U6MOLDtjm9M', 'type': 'tool_call'}, {'name': 'addadd', 'args': {'a': 11, 'b': 49}, 'id': 'call_97bvpojgqDbKFkZqgwbtJiYdcall_ItCryAXXxJRg9y3U4ikGJq4j', 'type': 'tool_call'}]
[{'name': 'multiplymultiplymultiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_6LhRibqh52pMYWDp6f2Wfy4Qcall_erQMMzwDObuV2U6MOLDtjm9Mcall_Js6iWl6ZqnaOsqHxTfzPhm0p', 'type': 'tool_call'}, {'name': 'addadd', 'args': {'a': 11, 'b': 49}, 'id': 'call_97bvpojgqDbKFkZqgwbtJiYdcall_ItCryAXXxJRg9y3U4ikGJq4j', 'type': 'tool_call'}]
[{'name': 'multiplymultiplymultiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_6LhRibqh52pMYWDp6f2Wfy4Qcall_erQMMzwDObuV2U6MOLDtjm9Mcall_Js6iWl6ZqnaOsqHxTfzPhm0p', 'type': 'tool_call'}, {'name': 'addadd', 'args': {'a': 11, 'b': 49}, 'id': 'call_97bvpojgqDbKFkZqgwbtJiYdcall_ItCryAXXxJRg9y3U4ikGJq4j', 'type': 'tool_call'}]
[{'name': 'multiplymultiplymultiply', 'args': {'a': 3, 'b': 12}, 'id': 'ca

In [16]:
print(type(gathered.tool_calls[0]["args"]))

<class 'dict'>


In [18]:
from langchain_core.messages import HumanMessage, ToolMessage


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rQaqPrGLUpTXGVYEarptxUMT', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_dD1nZDCwYY1btHTzbcUiVwtY', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 113, 'total_tokens': 163, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7e222217-db03-4f35-a790-21255030df88-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_rQaqPrGLUpTXGVYEarptxUMT', 'type': 'tool_call'}, {'name': 'add

In [19]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 12\\) is 36, and \\(11 + 49\\) equals 60.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 179, 'total_tokens': 207, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-35d56643-4e75-4ba3-8a1e-9be826563bb3-0', usage_metadata={'input_tokens': 179, 'output_tokens': 28, 'total_tokens': 207, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [20]:
llm_with_tools.invoke(
    """Whats 119 times 8 minus 20. Don't so any math yourself,
      only se tools for math . Respect of operations"""
).tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_pM25CsMSgAorf0zOWqY3ka4K',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': -20, 'b': 0},
  'id': 'call_o20L5JHAvc1W01CkO5CMb9Y1',
  'type': 'tool_call'}]

In [23]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_nUlhqFAJ6ey24XitYJvmEAbC',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 20, 'b': -20},
  'id': 'call_lEZW9w5K7LbAAaHOSfICBPNO',
  'type': 'tool_call'}]